#特征工程

生成特征：一个最基础的特征组合包括UI特征，U特征，I特征；特征的具体定义就是发挥你智慧的空间了


##I 品牌的特征 f1_xxx（哪些品牌会被买？）

##U 用户的特征 f2_xxx（哪些用户会买？） 

##UI 用户品牌的特征 f3_xxx（用户会买哪些品牌？）

###基本统计类特征

1. 截止到最后一天（比如交叉验证训练集的第61天）累计的点击（或衰减后点击）
2. 同样的特征分别再针对收藏、购物车、购买等操作进行统计
3. 截止到最后一天，用户对品牌进行过访问的次数、购买的次数、收藏的次数、购物车的次数（或经过时间衰减）
4. 截止到最后一天，用户对品牌进行第一次访问（收藏、购物车、购买）到最后一次访问的时间间隔（或经过时间衰减）
5. 用户最后一次访问（收藏、购物车、购买）距离分隔日期（如62）的时间间隔
    
###比值类特征

1. 购买该品牌次数/总购买次数
2. 对该品牌的总点击（购买、收藏、购物车）数/访问次数
3. 在对品牌A进行访问的那些天里，用户对A的点击数/那些天里的总点击数
4. 在对品牌A进行访问的那些天里，A的销量/那些天里的所有品牌的总销量（这个特征在实现思路上消耗内存太大，未能实现）##用户品牌的特征（用户会买哪些品牌？）
5. 用户访问品牌A的天数/用户总活跃天数
6. 用户购买品牌A的天数/用户总购买天数


用【用户购买该brand的天数/用户访问该brand的天数】来刻画用户对该品牌的忠诚度

用户购买品牌时的月份即用字符串表示，而用户购买品牌的次数则用数字表示

可以发现点击量越高的，点击转购买率也往往越低，尤其点击量极高的用户，往往购买量为0。由此可以考虑将用户购买量、点击转购买率等一系列特征用于描述用户是否会在未来一个月发生购买

统计 用户从初次访问品牌到最终购买品牌的时间，可以发现绝大多数购买都是当天接触当天完成，越往后用户购买的可能性越低。由此可以推测用户对品牌的购买意愿是随着距离上次访问的时间拉长而衰减的，进而可以使用衰减函数来模拟该购买意愿。

##Unique (U,I) index in train_set

In [1]:
import pandas as pd

from collections import defaultdict
import datetime

In [2]:
_features = {
    'f_behavior_count': {1:'f3_1', 2:'f3_2', 3:'f3_3', 4:'f3_4'},
    'f_behavior_lasttime_delta': {1:'f3_5', 2:'f3_6', 3:'f3_7', 4:'f3_8'},
}

##3. UI特征 （用户-商品特征）

###1 基本统计类特征

In [3]:
parseDate = lambda s: datetime.datetime.strptime(s, "%Y-%m-%d %H")
def f_behavior_count(df):
    '''截止到最后一天，用户对商品进行过访问的次数、购买的次数、收藏的次数、购物车的次数
    TODO: 加时间衰减
    @return dict; {'f3_1':browse_count, 'f3_2': collect_count, ...}
    '''
    features = {1:'f3_1', 2:'f3_2', 3:'f3_3', 4:'f3_4'}
    c = df.groupby('behavior_type').count()
    f = defaultdict(int)
    for b in c.index:
        f[features[b]] = c.loc[b][0]
    return [f['f3_1'],f['f3_2'], f['f3_3'], f['f3_4']]

def f_behavior_lasttime_delta(df, seperator=datetime.datetime(2014,12,18,0)):
    '''用户最后一次对商品访问（收藏、购物车、购买）距离分隔日期（如62）的时间间隔，如果没有发生动作，默认间隔60天
    @return dict; 
    '''
    features = {1:'f3_5', 2:'f3_6', 3:'f3_7', 4:'f3_8'}
    group_max = df.groupby('behavior_type').max()
    f = defaultdict(int)
    for v in features.values(): f[v] = 60
    for b in group_max.index:
        f[features[b]] = (seperator - parseDate(group_max.loc[b].time)).days
    return [f['f3_5'],f['f3_6'], f['f3_7'], f['f3_8']]

###2 比值类特征

1. 购买该品牌次数/总购买次数
2. 对该品牌的总点击（购买、收藏、购物车）数/访问次数
3. 在对品牌A进行访问的那些天里，用户对A的点击数/那些天里的总点击数
4. 在对品牌A进行访问的那些天里，A的销量/那些天里的所有品牌的总销量（这个特征在实现思路上消耗内存太大，未能实现）##用户品牌的特征（用户会买哪些品牌？）
5. 用户访问品牌A的天数/用户总活跃天数
6. 用户购买品牌A的天数/用户总购买天数

##构建train_set （训练集）

In [4]:
# raw_train_set = pd.read_csv('data/raw_train_set.csv')
# gp_ui = raw_train_set.groupby(['user_id', 'item_id'])

# columns = ['user_id', 'item_id']
# columns.extend(_features['f_behavior_count'].values())
# columns.extend(_features['f_behavior_lasttime_delta'].values())

# train_set = pd.DataFrame(columns=columns)
# for (u,i), gpdf in gp_ui:
#     row = [u,i]
#     row.extend(f_behavior_count(gpdf))
#     row.extend(f_behavior_lasttime_delta(gpdf))
#     train_set.loc[len(train_set)] = row

# train_set.to_csv('data/train_set.csv', index=False)

##Get train UI

In [5]:
def get_train_UI(train='data/raw_train_set.csv', to_file='data/train_set/ui_set.csv'): 
    raw_train_set = pd.read_csv(train)
    train_set = raw_train_set[['user_id', 'item_id']]

    gp_ui = train_set.groupby(['user_id', 'item_id'])
    ui_list = ( (u,i) for (u,i),gpdf in gp_ui)
    pd.DataFrame(ui_list, columns=['user_id', 'item_id']).to_csv(to_file, index=False)

In [6]:
# 1212
get_train_UI('data/1212raw_train_set.csv', 'data/train_set/1212ui_set.csv')
# # ~1212
# get_train_UI()

IOError: File data/1212raw_train_set.csv does not exist

##将训练集打上标签

In [ ]:
def tag(df, totag, fromtag, to_file='data/train_set.csv'):
    '''将训练集用result打上buy标记'''
    result_set = set((i[0],i[1]) for i in fromtag.values)
    #train_set = pd.DataFrame(train_set, 
    #   columns=['user_id', 'item_id', 'behavior_type', 'user_geohash', 'item_category', 'time', 'buy'])
    print 'to tag'
    df['buy'] = [ 1 if ui in result_set else 0 
        for ui, gpdf in totag
    ]
    if to_file:
        df.to_csv(to_file, index=False)
    return df

In [ ]:
def tag_of_train_set():
    '''get train_set/tag_set.csv'''
    result = pd.read_csv('data/result.csv')
    raw_train_set = pd.read_csv('data/1212raw_train_set.csv')
    train_set = raw_train_set[['user_id', 'item_id']]
    print 'result, train_set loaded'

    gp_ui = train_set.groupby(['user_id', 'item_id'])
    print 'groupby ok'

    columns = ['buy']
    tag_set = pd.DataFrame(columns=columns)

    tag(tag_set, gp_ui, result, to_file='data/train_set/1212tag_set.csv')
    print 'done'

##Merge train features to train_set

In [ ]:
# def merge_features_to_train_set():
#     df_tag = pd.read_csv('data/train_set/tag_set.csv')
#     df_f3_1to4 = pd.read_csv('data/train_set/feature_set_behavior_count.csv')
#     df_f3_5to8 = pd.read_csv('data/train_set/feature_set_behavior_lasttime_delta.csv')
#     df_f2_1to2 = pd.read_csv('data/train_set/feature_u_count.csv')
#     pd.concat([df_tag, df_f3_1to4, df_f3_5to8, df_f2_1to2], axis=1)\
#     .to_csv('data/train_set/train_set.csv', index=False)
    
def merge_features_to_train_set1212():
#     df_tag = pd.read_csv('data/train_set/1212tag_set.csv')
#     df_f3_1to4 = pd.read_csv('data/train_set/1212feature_set_behavior_count.csv')
#     df_f3_5to8 = pd.read_csv('data/train_set/1212feature_set_behavior_lasttime_delta.csv')
#     df_f2_1to2 = pd.read_csv('data/train_set/1212feature_u_count.csv')
#     df_f1 = pd.read_csv('data/train_set/1212feature_i_count.csv')
    pd.concat([
            pd.read_csv('data/train_set/1212tag_set.csv'),
            pd.read_csv('data/train_set/1212feature_set_behavior_count.csv'),
            pd.read_csv('data/train_set/1212feature_set_behavior_lasttime_delta.csv'),
            pd.read_csv('data/train_set/1212feature_u_count.csv'),
            pd.read_csv('data/train_set/1212feature_i_count.csv'),
        ], axis=1).to_csv('data/train_set/1212train_set.csv', index=False)

In [ ]:
# TODO 分1212
tag_of_train_set()

## 1212
#merge_features_to_train_set1212()

# ～1212
# merge_features_to_train_set()

##构建test_set (测试集)

##Get test UI

In [ ]:
def get_test_UI():    
    test_set = pd.read_csv('data/test_set/tianchi_mobile_recommend_test_user.csv')
    test_set = test_set[['user_id', 'item_id']]
    print 'test set loaded'
    # P商品子集
    P = pd.read_csv('data/raw/tianchi_mobile_recommend_train_item.csv')
    P_item_set = set(P.item_id.values)
    # 过滤出 item_id 属于 P(商品子集) 的记录
    test_set = test_set[test_set.item_id.map(lambda x: x in P_item_set)]

    gp_ui = test_set.groupby(['user_id', 'item_id'])
    ui_list = ( (u,i) for (u,i),gpdf in gp_ui)
    pd.DataFrame(ui_list, columns=['user_id', 'item_id']).to_csv('data/test_set/1212ui_set.csv', index=False)

##Merge test features to test_set

In [ ]:
# def merge_features_to_test_set():
#     ui_set = pd.read_csv('data/test_set/ui_set.csv')
#     df_f3_1to4 = pd.read_csv('data/test_set/feature_set_behavior_count.csv')
#     df_f3_5to8 = pd.read_csv('data/test_set/feature_set_behavior_lasttime_delta.csv')
#     df_f2_1to2 = pd.read_csv('data/test_set/feature_u_count.csv')
#     pd.concat([ui_set, df_f3_1to4, df_f3_5to8, df_f2_1to2], axis=1).to_csv('data/test_set/test_set.csv', index=False)
    
def merge_features_to_test_set1212():
#     ui_set = pd.read_csv('data/test_set/1212ui_set.csv')
#     df_f3_1to4 = pd.read_csv('data/test_set/1212feature_set_behavior_count.csv')
#     df_f3_5to8 = pd.read_csv('data/test_set/1212feature_set_behavior_lasttime_delta.csv')
#     df_f2_1to2 = pd.read_csv('data/test_set/1212feature_u_count.csv')
#     df_f1 = pd.read_csv('data/test_set/1212feature_i_count.csv')
    pd.concat([
            pd.read_csv('data/test_set/1212ui_set.csv'),
            pd.read_csv('data/test_set/1212feature_set_behavior_count.csv'),
            pd.read_csv('data/test_set/1212feature_set_behavior_lasttime_delta.csv'),
            pd.read_csv('data/test_set/1212feature_u_count.csv'),
            pd.read_csv('data/test_set/1212feature_i_count.csv'),
        ], axis=1).to_csv('data/test_set/1212test_set.csv', index=False)

In [ ]:
get_test_UI()

## 1212
#merge_features_to_test_set1212()

# #~1212
# merge_features_to_test_set()